In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q timm pytorch-metric-learning[with-hooks]

     |████████████████████████████████| 549 kB 4.8 MB/s 
     |████████████████████████████████| 111 kB 74.7 MB/s 
     |████████████████████████████████| 182 kB 78.0 MB/s 
     |████████████████████████████████| 85.5 MB 85 kB/s 


In [3]:
import pandas as pd

import statistics

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

import pytorch_metric_learning
from pytorch_metric_learning.utils.inference import InferenceModel

In [4]:
# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# args
image_size = 224
k=10

In [6]:
class TestTransforms():

    def __init__(self, image_size):
        
        self.data_transform = {
            'test': transforms.Compose([
                transforms.Resize((image_size, image_size)),
                transforms.ToTensor(),
                transforms.Normalize(
                    [0.485, 0.456, 0.406],
                    [0.229, 0.224, 0.225]
                    ),
                ]),
        }
    
    def __call__(self, phase, img):
        return self.data_transform[phase](img)

data

In [7]:
# read data
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/pml_defect/input/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/pml_defect/input/sample_submission.csv', header=None)

# train image name list & label list
image_name_list = train['id']
label_list = train['target']

# index2id & index2target
index2id = train['id'].to_dict()
index2target = train['target'].to_dict()

# test image name list & dummy label list
x_test = test[0].values
dummy = test[0].values

dataset, dataloader

In [8]:
import os
from PIL import Image
from torch.utils.data import Dataset


class DefectDataset(Dataset):
    def __init__(self, image_name_list, label_list, img_dir, transform=None, phase=None):
        self.image_name_list = image_name_list
        self.label_list = label_list
        self.img_dir = img_dir
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.image_name_list)

    def __getitem__(self, index):
        image_path = os.path.join(self.img_dir, self.image_name_list[index])
        image = Image.open(image_path)
        image = self.transform(self.phase, image)
        label = self.label_list[index]
        
        return image, label

# dataset
dataset = DefectDataset(
    image_name_list,
    label_list,
    img_dir='/content/drive/MyDrive/Colab Notebooks/pml_defect/input/train_data',
    transform=TestTransforms(image_size=image_size),
    phase='test'
)

# test dataset
test_dataset = DefectDataset(
    x_test,
    dummy,
    img_dir='/content/drive/MyDrive/Colab Notebooks/defect/input/test_data',
    transform=TestTransforms(image_size=image_size),
    phase='test'
)

In [9]:
# dataloader
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

model

In [10]:
from timm.models.registry import model_entrypoint
import torch
import torch.nn as nn
import timm
import pytorch_metric_learning
from pytorch_metric_learning.utils import common_functions


class ConvnextBase(nn.Module):
    def __init__(self, pretrained, embedding_size):
        super(ConvnextBase, self).__init__()
        self.pretrained = pretrained
        self.embedding_size = embedding_size

        self.trunk = timm.create_model(
            'convnext_base',
            pretrained=self.pretrained,
        )
        self.trunk.head.fc = common_functions.Identity()
        self.embedder = nn.Linear(1024, self.embedding_size)
    
    def forward(self, x):
        x = self.trunk(x)
        x = self.embedder(x)
        return x


model = ConvnextBase(
    pretrained=False,
    embedding_size=512,
)
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/pml_defect/model/clear-armadillo-6.pth', map_location=torch.device(device)))
model.to(device)

ConvnextBase(
  (trunk): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
            (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (fc2): Linear(in_features=512, out_features=128, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 

inference

In [11]:
im = InferenceModel(model)

In [12]:
im.train_knn(dataset)

In [15]:
from tqdm import tqdm

filenames, distances, preds, top1s, modes = [], [], [], [], []
with torch.no_grad():
    for image, filename in test_dataloader:
        # distance & index of dataset
        distance, index = im.get_nearest_neighbors(image, k=k)
        distance = distance.cpu().numpy()

        # index => target
        pred = [index2target[int(k)] for k in index.cpu().numpy().squeeze()]

        # top1 pred
        top1 = pred[0]

        # mode of topK
        mode = statistics.mode(pred)
        
        filenames.extend(filename)
        distances.extend(distance)
        preds.append(pred)
        top1s.append(top1)
        modes.append(mode)

    df = pd.DataFrame({"filename": filenames, "distance": distances, "preds": preds, "top1": top1s, "mode": modes})

In [14]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/pml_defect/submit/inderence_test.csv', sep=',', header=None, index=None)